In [1]:
import gymnasium as gym
import random
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [2]:
#visualise maze:
mazeSize = [4,12]

statePositions = [[] for _ in range(mazeSize[0])]
stateNum = 0
for i in range(mazeSize[0]):
    for j in range(mazeSize[1]):
        statePositions[i].append(stateNum)
        stateNum += 1
        

        
giftState = 47
            
print(giftState)
print(statePositions)

env = gym.make("CliffWalking-v0")

47
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]]


In [49]:
qTable_1 = {}
def resetTable():
    global qTable_1
    qTable_1 = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        qTable_1[i] = [0,0,0,0] 
    global currentState
    currentState = 36
    
def getPosition(state):
    for i in range(len(statePositions)):
        for j in range(len(statePositions[i])):
            if statePositions[i][j] == state:
                return i, j
            
def calcReward(state, nextState):
    y1, x1 = getPosition(state)
    y2, x2 = getPosition(nextState)
    y3, x3 = getPosition(giftState)
    
    currentDist = (((y3 - y1)**2)+((x3 - x1)**2))**0.5
    nextDist = (((y3 - y2)**2)+((x3 - x2)**2))**0.5
    
    changeInDist = currentDist-nextDist
    return changeInDist/2

def nextStep(state):
    global qTable_1
    possMoves = calcPossibleMoves(state)
    
    if random.random() < epsilonValue:
        nextMove = random.choice(possMoves)
    else:
        qValues = {}
        for move in possMoves:
            qValues[move] = qTable_1[state][move]

        maxValue = max(qValues.values())
        minValue = min(qValues.values())
        count_max = sum(1 for value in qValues.values() if value == maxValue)
        count_min = sum(1 for value in qValues.values() if value == minValue)

        if count_max > 1 and count_max < len(possMoves):
            nextMove = random.choice([move for move in possMoves if qValues[move] != minValue])
        elif count_max == len(possMoves):
            nextMove = random.choice(possMoves)
        else:
            nextMove = max(qValues, key=qValues.get)
    return nextMove

def calcPossibleMoves(state):
    global qTable_1
    possibleMoves = []
    
    if state == 36:
        return [0,1]
    
    if (state+1) % mazeSize[1] != 0:
        possibleMoves.append(1)
        
    if (state+1) % mazeSize[1] != 1:
        possibleMoves.append(3)
        
    if state > (mazeSize[1]-1):
        possibleMoves.append(0)
    
    if state < ((mazeSize[0] * mazeSize[1]) - mazeSize[1]):
        possibleMoves.append(2)
        
    return possibleMoves


def pathFound():
    state = 36
    for i in range(mazeSize[0]*mazeSize[1]-1):
        bestDirection = None
        if max(qTable_1[state]) > 0:
            bestDirection = qTable_1[state].index(max(qTable_1[state]))
        newState = state
        if bestDirection == 0 and 0 in calcPossibleMoves(state):
            newState = state - 1
        elif bestDirection == 1 and 1 in calcPossibleMoves(state):
            newState = state + mazeSize[1]
        elif bestDirection == 2 and 2 in calcPossibleMoves(state):
            newState = state + 1
        elif bestDirection == 3 and 3 in calcPossibleMoves(state):
            newState = state - mazeSize[1]

        if newState == giftState:
            return True
        state = newState
    return False

timesVisited = {}
def resetVisited():
    global timesVisited
    timesVisited = {}
    for i in range(mazeSize[0]*mazeSize[1]):
        timesVisited[i] = 0

def updateTable_qLearning(direction, nextState, reward, foundPath):
    global qTable_1
    global currentState
    didConverge = False
    updated = qTable_1[currentState][direction] + alpha*(reward + (max(qTable_1[nextState])) - qTable_1[currentState][direction])
    changeInQ = round(abs(qTable_1[currentState][direction] - updated),5)
    if changeInQ < convergenceThresh and changeInQ > 0:
        if foundPath:
            didConverge = True
    qTable_1[currentState][direction] = updated
    timesVisited[currentState] += 1
    currentState = nextState
    return didConverge, changeInQ

In [50]:
epsilonValue = 0.35
alpha = 0.65
convergenceThresh = 0.001
revistPenalty = -0.25
totalTrials = 0
currentState = 36

In [53]:
def qLearning(maxEpisodes, currentTrial):
    global currentState
    global revisitPenalty
    global totalTrials
    global totalTime_start
    
    firstPath = []
    firstPathEpisode = []
    foundFirstPath = False

    currentEpisode = 1
    converged = False

    resetVisited()
    resetTable()
    env.reset()
    
    start_time = time.time()
    
    while currentEpisode <= maxEpisodes:
        if converged:
            break

        direction = nextStep(currentState)
        if not foundFirstPath:
            firstPath.append([currentState, direction])
        if currentState != 47:
            nextState, reward, terminated, truncated, info = env.step(direction)
        
        reward = 0
        if terminated:
                if nextState == 47:
                    reward = 10
                    if not foundFirstPath:
                        firstPathEpisode = [currentEpisode, firstPath]
                        foundFirstPath = True
                else:
                    reward = -1
        if not terminated:
            reward = calcReward(currentState, nextState)
            if timesVisited[nextState] > 0:
                reward += revistPenalty*timesVisited[nextState]
        
        converged, changeInQ = updateTable_qLearning(direction, nextState, reward, foundFirstPath)

        if terminated or truncated or converged:
            observation, info = env.reset()
            currentState = 36
            if not converged:
                currentEpisode += 1
            if not foundFirstPath:
                firstPath = []
            resetVisited()


        if converged:
            end_time = time.time()
        
        time.sleep(0.002)
        clear_output(wait=True)
        if totalTrials != 0:
            print("Trial: " + 'Q-Learning, ' + 'trialType '+ str(trialType) + ', Trial ' + str(currentTrial) + "/" + str(totalTrials))
        print("Episode: " + str(currentEpisode) + "/" + str(maxEpisodes))
        print("Trial Time: " + str(round(time.time()-start_time, 3)) + " sec")
        print(currentState)
        if totalTrials != 0:
            print("Total Time: " + str(round((time.time()-totalTime_start)/60, 3)) + " min")
        print("Q-Table:")
        for i in range(len(qTable_1)):
            print(str(i) + ": " + str(qTable_1[i]))
        print("change in Q: " + str(changeInQ))
        
    duration = 0
    if converged:
        duration = end_time - start_time
        print(str(round(duration, 3)) + " seconds to converge")
    else:
        duration = None
        print("No convergence")
        
    return (currentEpisode-1), duration, firstPathEpisode

In [56]:
episodes, duration, firstPath = qLearning(2000,1)

Episode: 6/2000
Trial Time: 1.141 sec
36
Q-Table:
0: [0, 0.7306347083845248, 0, 0]
1: [0, 0.6433295264742394, 0, -5.79557263281294e-05]
2: [0, 0.7400526980366546, -0.7658875541037989, -0.3098788967317591]
3: [0, 0.9090763751634257, -0.7752423431971139, -0.6451186510051807]
4: [0, 1.1380736528910773, -0.5596027637240374, -0.3016749578224271]
5: [0, 1.059462407237795, -0.4324929551331752, -0.45745998134347526]
6: [0, 0.8388624945414902, 0, -0.28510691223757234]
7: [0, 0.7572196869787384, 0.18716636329582798, 0]
8: [0, -0.07698789843471257, 0.6775007648807323, 0]
9: [0, 0, 0.45876414924023706, -0.538228244403053]
10: [0, 0, 0, 0.013087644560225481]
11: [0, 0, 0, 0]
12: [-0.0719596681350398, 0.4357213326642657, 0, 0]
13: [0.01705935131502435, -0.7184814510385465, -0.047568825790096435, -0.274236722860647]
14: [-0.1663848399325755, -0.7737419902403947, -0.7582798628605907, -0.48493770222910115]
15: [-0.8723884242473658, -0.14369046150105097, -0.4257364103339635, -0.752049944157757]
16: [0.1

In [58]:
episodes

5